In [1]:
from Explanations_Models.Custom_DT.LIME import LIME
from FeatureImportance.FI import FeatureImportance
import yaml
import argparse
import os
import sys
import numpy as np
import torch
import gym
import matplotlib
import matplotlib.pyplot as plt
import unittest
from DeepLearning_Models.utils.general import join, plot_combined
from DeepLearning_Models.ActorCritic.policy_gradient import PolicyGradient
from EnvRunner import GymRunner
import json
import warnings
from tqdm import tqdm

# Suppress all deprecation warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

yaml.add_constructor("!join", join)
parser = argparse.ArgumentParser()

parser.add_argument("--config_filename", required=False, type=str)

_StoreAction(option_strings=['--config_filename'], dest='config_filename', nargs=None, const=None, default=None, type=<class 'str'>, choices=None, help=None, metavar=None)

In [2]:

config_file = open("config_envs/{}.yml".format("_bipedal"))
config = yaml.load(config_file, Loader=yaml.FullLoader)
config.update(yaml.load(open("config_explanations/{}.yml".format("_bipedal"), encoding="utf8"), Loader= yaml.FullLoader))
Runner = GymRunner(config)
Runner.load_weights(PATH = None)

device:  cpu


c:\Users\blav1\Scripts\BryansClaimToFame\InterpretableModelExploration\EnvRunner.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.baseline_network.load_state_

In [3]:
config["surrogate"]["use_FI"] = True
FI = FeatureImportance(config, Runner.model.network)
DTMode = LIME(config, Runner, FI)
#DTMod = LIME(config, Runner)
thang = []
# for i in range(0, 10):
#     DTMode.train()
#     thang.append(DTMode.percent_Correct())
#     print(np.mean(thang), np.var(thang))


In [4]:
thang2 = []
config["surrogate"]["use_FI"] = False
DTMod = LIME(config, Runner)

for i in range(0, 10):
    DTMod.train()
    thang2.append(DTMod.percent_Correct())
thang2 = np.array(thang2)
print(thang2.mean(), thang2.var())

1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
2
1
1
1
1
1
3
1
1
2
1
1
1
1
1
1
1
1
1
1
2
1
1
2
1
1
1
1
1
2
1
1
1
1
1
1
1
1
1
1
2
1
1
1
1
1
1
1
1
2
1
1
1
2
3.1965623 0.7671832


ClassForgetting

In [9]:
#DTMod = LIME(config, Runner)
thang = []
for i in range(0, 10):
    DTMode.train()
    thang.append(DTMode.percent_Correct())
    print(np.mean(thang), np.var(thang))

0.9 0.0
0.7250000000000001 0.030624999999999996
0.7800000000000001 0.026466666666666663
0.7625000000000001 0.02076875
0.7090000000000001 0.028064
0.7291666666666666 0.025420138888888888
0.745 0.02329285714285714
0.764375 0.023008984375
0.78 0.022405555555555554
0.7925000000000001 0.02157125


what we got

In [10]:
thang1 = np.array(thang)
print(thang1.mean(), thang1.var())

0.7925000000000001 0.02157125


what we beat

In [11]:
thang2 = []
config["surrogate"]["use_FI"] = False
DTMod = LIME(config, Runner)

for i in range(0, 10):
    DTMod.train()
    thang2.append(DTMod.percent_Correct())

In [12]:
thang2 = np.array(thang2)
print(thang2.mean(), thang2.var())

0.7064999999999999 0.01735025


In [7]:
from scipy.stats import ttest_ind
t_statistic, p_value = ttest_ind(thang1, thang2)
print("T-statistic:", t_statistic)
print("P-value:", p_value)

NameError: name 'thang2' is not defined

In [8]:

out, matrix = FI.Calc_Feature_Importance(torch.tensor([1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,1.,0.,]), printer=True)

point tensor([1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0.,
        1., 0., 1., 0., 1., 0.])
output tensor([ 1.5826, -7.8118,  2.3578, -2.1897], grad_fn=<ViewBackward0>)
relevence [tensor([-0.4360,  0.0032, -0.2563,  0.0032, -0.5618,  0.0032,  0.5331,  0.0032,
        -0.1117,  0.0032,  0.4333,  0.0032,  0.4067,  0.0032,  0.5228,  0.0032,
         0.7383,  0.0032,  0.0690,  0.0032,  0.5256,  0.0032, -0.3183,  0.0032],
       grad_fn=<SqueezeBackward4>), tensor([ 4.6117, -0.0781,  0.9442, -0.0781,  5.6282, -0.0781, -3.1760, -0.0781,
         1.1894, -0.0781, -3.2217, -0.0781, -2.6642, -0.0781, -2.1719, -0.0781,
        -5.1672, -0.0781, -1.4828, -0.0781, -4.0037, -0.0781,  2.6397, -0.0781],
       grad_fn=<SqueezeBackward4>), tensor([-0.3696,  0.0232, -0.4396,  0.0232, -0.2606,  0.0232,  0.5377,  0.0232,
        -0.0442,  0.0232,  0.3225,  0.0232,  0.5381,  0.0232,  0.5866,  0.0232,
         0.5562,  0.0232,  0.2230,  0.0232,  0.6054,  0.0232, -0.1759,  0.0232]

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 3 dimensions. The detected shape was (2, 10, 4) + inhomogeneous part.

In [1]:
print(FI.shape)

NameError: name 'FI' is not defined

In [1]:
import pandas as pd

columns = pd.MultiIndex.from_product([config["picture"]["class_names"], config["picture"]["labels"]], names=['OutLogit', 'InLogit'])

# Reshape the data to (60, 8) to match the multi-level column structure
reshaped_data = FI.reshape(FI.shape[0], -1)

# Create the DataFrame
FI = pd.DataFrame(reshaped_data, columns=columns)
out = pd.DataFrame(out, columns=config["picture"]["class_names"])

NameError: name 'matrix' is not defined

In [3]:
DT = LIME(config, Runner)

In [ ]:
def print_activation(layer, input, output):
    print(f"Layer: {layer}, Activation Output: {output}")

# Register hooks to each layer
for layer in model.children():
    layer.register_forward_hook(print_activation)

# Dummy input to pass through the model
dummy_input = torch.randn(1, 10)  # Adjust size to match input layer
model(dummy_input)

In [5]:
Runner.model.network.forward(torch.tensor([[4.,4.,4.,4.,]]))


tensor([[-53.5020,  57.9378]], grad_fn=<AddmmBackward0>)

In [14]:
X,Y = DT.train(returner=True)


60


In [15]:
DT.percent_Correct(print_val=True)


1.1668402

In [16]:
DT.uniform_Correct(print_val=True)

111.059525

In [9]:
print(DT.surr_model.model.max_depth)
DT.surr_model.model.get_dict_representation()

10


{'Feature': 'Hip Joint 1 Speed',
 'Bucket': 0.5963983535766602,
 'Left_Child': {'Feature': 'Knee Joint 1 Angle',
  'Bucket': -0.16023868322372437,
  'Left_Child': {'Feature': 'Knee Joint 2 Angle',
   'Bucket': -0.030735313892364502,
   'Left_Child': {'Feature': 'Velocity X',
    'Bucket': 0.5160324573516846,
    'Left_Child': {'Feature': 'Hull Angle',
     'Bucket': -0.06656686216592789,
     'Left_Child': {'Value': array([-2.6837106 , -0.3007074 ,  1.4836516 ,  0.03225509], dtype=float32)},
     'Right_Child': {'Feature': 'Hull Angle',
      'Bucket': 0.0036161281168460846,
      'Left_Child': {'Value': array([-2.7936342 ,  2.177329  ,  1.6616061 ,  0.34213853], dtype=float32)},
      'Right_Child': {'Value': array([-1.1121528,  2.3536325,  2.649001 ,  1.4963849], dtype=float32)}}},
    'Right_Child': {'Feature': 'Hull Angle',
     'Bucket': 0.08883405476808548,
     'Left_Child': {'Feature': 'Hull Angle',
      'Bucket': 0.06528535485267639,
      'Left_Child': {'Value': array([-3.00

In [9]:
np.mean(Y, axis=0)

array([ 0.74338526,  1.4490379 ,  1.0475755 , -0.23259966], dtype=float32)

In [ ]:
from scipy.spatial.distance import pdist
import pandas as pd
Y = pd.DataFrame(Y, columns=config["picture"]["class_names"])
np.mean(pdist(Y.values, metric="euclidean"))

In [9]:
import pandas as pd
Y = np.array([[1., 4.2, 3.3]])
Y = pd.DataFrame(Y, columns = ["hit1", "hit2", "hit3"])

In [ ]:
from scipy.spatial.distance import pdist

np.mean(pdist(Y.values, metric="euclidean"))

In [31]:
a = torch.tensor([1., 2., 3.])
W = torch.tensor([[2., 3.,],[2., 3.,],[2., 3.,]])
b = torch.tensor([.5,.6])

In [32]:
print(a.shape, W.shape, b.shape)

torch.Size([3]) torch.Size([3, 2]) torch.Size([2])


In [33]:
a*W

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1

In [34]:
a@W+b

tensor([12.5000, 18.6000])

In [35]:
norm = []
CorVec = []
for i in range(len(b)):
    tmp = a*W[:,i] + b[i]
    norm.append(tmp.sum())
    CorVec.append(tmp)
    print(a*W[:,i] + b[i])
norm = torch.stack(norm)
CorVec = torch.stack(CorVec)
print(norm, CorVec)

tensor([2.5000, 4.5000, 6.5000])
tensor([3.6000, 6.6000, 9.6000])
tensor([13.5000, 19.8000]) tensor([[2.5000, 4.5000, 6.5000],
        [3.6000, 6.6000, 9.6000]])


In [36]:
norm += 1e-9

In [37]:
obj = torch.relu(a@W+b)

In [38]:
obj

tensor([12.5000, 18.6000])

In [39]:
normalized_CorVec = CorVec / norm.unsqueeze(1)

In [40]:
normalized_CorVec

tensor([[0.1852, 0.3333, 0.4815],
        [0.1818, 0.3333, 0.4848]])

In [41]:
out = obj @ normalized_CorVec
print(out)

tensor([ 5.6966, 10.3667, 15.0367])
